In [1]:
import os
import collections

import itertools
import tqdm
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from kolmov import crossval_table, get_color_fader

import tensorflow as tf

Welcome to JupyROOT 6.16/00


2022-01-09 23:27:49.198581: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...


## Create a dictionary

Since the output of saphyra is like a dictionary we need to navigate on and get all information.

kolmov has a class called crossval_table which allow us to get this information and tranform into a pandas Dataframe.

The first thing to do is define a OrderedDict to access all information inside of saphyra tuned file.

In [2]:
def create_op_dict(op):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : "reference/"+op+"/pd_ref#0",
              'fa_ref'    : "reference/"+op+"/fa_ref#0",
              'sp_ref'    : "reference/"+op+"/sp_ref",
              'pd_val'    : "reference/"+op+"/pd_val#0",
              'fa_val'    : "reference/"+op+"/fa_val#0",
              'sp_val'    : "reference/"+op+"/sp_val",
              'pd_op'     : "reference/"+op+"/pd_op#0",
              'fa_op'     : "reference/"+op+"/fa_op#0",
              'sp_op'     : "reference/"+op+"/sp_op",

              # Counts
              'pd_ref_passed'    : "reference/"+op+"/pd_ref#1",
              'fa_ref_passed'    : "reference/"+op+"/fa_ref#1",
              'pd_ref_total'     : "reference/"+op+"/pd_ref#2",
              'fa_ref_total'     : "reference/"+op+"/fa_ref#2",
              'pd_val_passed'    : "reference/"+op+"/pd_val#1",
              'fa_val_passed'    : "reference/"+op+"/fa_val#1",
              'pd_val_total'     : "reference/"+op+"/pd_val#2",
              'fa_val_total'     : "reference/"+op+"/fa_val#2",
              'pd_op_passed'     : "reference/"+op+"/pd_op#1",
              'fa_op_passed'     : "reference/"+op+"/fa_op#1",
              'pd_op_total'      : "reference/"+op+"/pd_op#2",
              'fa_op_total'      : "reference/"+op+"/fa_op#2",
    })
    return d


op_names = ['tight', 'medium', 'loose', 'vloose']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op)

In [3]:
etbins  = [4, 7, 10, 15]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.47]

l_path  = '/home/micael/Documents/NeuralRinger/Jpsiee'

tunes_path    = os.path.join(l_path, 'tunes')
analysis_path = os.path.join(l_path, 'analysis')

print('paths setted: \n -> tunes: %s \n -> analysis: %s' %(tunes_path, analysis_path))

paths setted: 
 -> tunes: /home/micael/Documents/NeuralRinger/Jpsiee/tunes 
 -> analysis: /home/micael/Documents/NeuralRinger/Jpsiee/analysis


## Initialize the crossval_table object

In this step we initialiaze the crossval_table object and fill with data from our training.

In [4]:
m_cv = crossval_table( config_dict=tuned_info, etbins = etbins , etabins = etabins )
m_cv.fill( os.path.join(tunes_path, 'v1/*/*/*.npz'), 'v1.r0')
#m_cv.fill( os.path.join(tunes_path, 'v1/r1/*/*/*.pic.gz'), 'v1.r1')

Reading v1.r0...: 100%|██████████| 3000/3000 [00:05<00:00, 510.02it/s]


2022-01-09 23:28:04,666 | Py.crossval_table                       INFO Reading file for v1.r0 tag from /home/micael/Documents/NeuralRinger/Jpsiee/tunes/v1/*/*/*.npz
2022-01-09 23:28:04,666 | Py.crossval_table                       INFO There are 3000 files for this task...
2022-01-09 23:28:04,666 | Py.crossval_table                       INFO Filling the table... 
2022-01-09 23:28:10,610 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


In [6]:
n_min, n_max = 2, 5
model_add_tag = { idx : '.mlp%i' %(neuron) for idx, neuron in enumerate(range(n_min, n_max +1))}
# add a sufix in train_tag
m_cv.table().train_tag = m_cv.table().train_tag + m_cv.table().model_idx.replace(model_add_tag)
m_cv.to_csv(os.path.join(analysis_path, 'v1/r0/table_v1.csv'))


In [5]:
best_inits = m_cv.filter_inits("max_sp_val")
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
1310,v1.r0,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.937434,...,28455,215903,2783,2437,2846,21590,27827,23588,28455,215903
1309,v1.r0,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.937434,...,28455,215903,2799,2708,2846,21590,27987,27206,28455,215903
1308,v1.r0,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.937434,...,28455,215903,2799,2708,2846,21590,27987,27206,28455,215903
1311,v1.r0,0,0,0,0,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.937434,...,28455,215903,2815,3897,2846,21590,28149,33945,28455,215903
1138,v1.r0,0,0,0,1,1,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.940628,...,28455,215903,2783,2490,2846,21590,27827,24306,28455,215903


In [6]:
len(best_inits), 15*10*4*4

(2400, 2400)

In [8]:
best_inits.model_idx.unique()

array([0, 1, 2, 3])

In [9]:
# since take a long time to open those files let's save into a .csv
print(analysis_path)
best_inits.to_csv(os.path.join(analysis_path, 'v1/r0/best_inits.csv'))

/home/micael/Documents/NeuralRinger/Jpsiee/analysis


In [10]:
print(analysis_path)
r0_path = 'v1/r0'

/home/micael/Documents/NeuralRinger/Jpsiee/analysis


In [11]:
map_key_dict ={
   'max_sp_val'    : (r'$SP_{max}$ (Test)', 'sp'),
   'max_sp_pd_val' : (r'$P_D$ (Test)', 'pd'),
   'max_sp_fa_val' : (r'$F_A$ (Test)', 'fa'),
   #'auc_val'       : (r'AUC (Test)', 'auc'),
}

from kolmov.utils.constants import str_etbins_jpsiee, str_etabins
# using as simple function in order to make easier plot all need measures
def create_cool_catplot(df, key, kind, mapped_key, output_name, tuning_flag, tuning_folder, list_of_neuros=None):
    # create the box plot. 
    # rename the columns names.
    # map the model idx into real # neurons.
    
    if list_of_neuros is None:
        list_of_neuros = range(2, 20+1)
    g = sns.catplot(data=(df
                        .replace({'model_idx' : {i :  n for i, n in zip(range(0,df.model_idx.max()+1),
                        range(2,20+1))},
                                'et_bin'    : {i : str_etbins_jpsiee[i] for i in range(3)},
                                'eta_bin'   : {i : str_etabins[i] for i in range(5)}})
                        .rename({'model_idx'  : '# Neurons',
                                'et_bin'     : r'$E_T$',
                                'eta_bin'    : r'$\eta$',
                                key : mapped_key},
                        axis=1)), x='# Neurons',
                        y=mapped_key, col=r'$\eta$', 
                        row=r'$E_T$', kind=kind, sharey=False,
                        )

    plt.tight_layout()
    plt.savefig(os.path.join(analysis_path, '%s/plots/%s_plot_%s_%s.png' %(tuning_folder, kind, output_name, tuning_flag)), dpi=150, facecolor='white')
    plt.close()

def create_cool_scatterplot(df, key1, key2, mapped_key1, mapped_key2, output_name, tuning_flag, tuning_folder):
    
    sns.relplot(data=(best_inits.replace({'model_idx' : {i :  n for i, n in zip(best_inits.model_idx.unique(), [2, 5, 10, 15, 20])},
                                          'et_bin'    : {i : str_etbins_jpsiee[i] for i in range(3)},
                                          'eta_bin'   : {i : str_etabins[i] for i in range(5)}})
                      .rename({'model_idx'  : '# Neurons',
                               'et_bin'     : r'$E_T$',
                               'eta_bin'    : r'$\eta$',
                               key1         : mapped_key1,
                               key2         : mapped_key2}, axis=1)),
                x=mapped_key1, y=mapped_key2, 
                palette=['red', 'orange', 'green'], style='# Neurons',
                hue='# Neurons', row=r'$E_T$', col=r'$\eta$', facet_kws=dict(sharex=False, sharey=False))
    
    plt.tight_layout()
    plot_path = os.path.join(analysis_path, '%s/plots' %tuning_folder)
    os.makedirs(plot_path, exist_ok=True)
    plt.savefig(os.path.join(analysis_path, '%s/scatter_plot_%s_%s.png' %(plot_path, output_name, tuning_flag)), dpi=150, facecolor='white')
    plt.close()

In [12]:
best_inits.groupby(['et_bin', 'eta_bin', 'sort', 'train_tag'])['sp_ref'].value_counts()

et_bin  eta_bin  sort  train_tag   sp_ref  
0       0        0     v1.r0.mlp2  0.824113    2
                                   0.817758    1
                                   0.832723    1
                       v1.r0.mlp3  0.824113    2
                                   0.817758    1
                                              ..
2       4        9     v1.r0.mlp4  0.648809    1
                       v1.r0.mlp5  0.570147    1
                                   0.620786    1
                                   0.636231    1
                                   0.648809    1
Name: sp_ref, Length: 2240, dtype: int64

In [13]:
ikey         = 'max_sp_val'
map_k, o_name = map_key_dict[ikey]

for ikind in ['box', 'violin', 'boxen']:
    # just get an arbitrary op_name to filter duplicats numbers
    aux_df = best_inits.loc[best_inits.op_name == 'tight'].copy()
    create_cool_catplot(df=aux_df, key=ikey, mapped_key=map_k, 
                        kind=ikind, output_name=o_name, tuning_flag='v1.r0.all_neurons', tuning_folder=r0_path)

In [14]:
# select some models to filter
selected_models = ['v1.r0.mlp%i' %(ineuron) for ineuron in [2, 5]]#, 10, 15, 20]]
print(selected_models)

['v1.r0.mlp2', 'v1.r0.mlp5']


In [15]:
best_inits[best_inits.train_tag.isin(selected_models)].train_tag.unique()

array(['v1.r0.mlp2', 'v1.r0.mlp5'], dtype=object)

In [16]:
for ikey in map_key_dict.keys():
    map_k, o_name = map_key_dict[ikey]
    for ikind in ['box', 'violin', 'boxen']:
        # just get an arbitrary op_name to filter duplicats numbers
        aux_df = best_inits.loc[best_inits.op_name == 'tight'].copy()
        create_cool_catplot(df=aux_df[aux_df.train_tag.isin(selected_models)], key=ikey, mapped_key=map_k,
                            kind=ikind, output_name=o_name, tuning_flag='v1.r0.selected_neurons', tuning_folder=r0_path)

## Filter the initializations and get the best sort

To get the best initialization in each sort and the best sort for each model configuration is easy since we are using pandas.


In [ ]:
for iet in best_inits['et_bin'].unique():
    iet_mask = best_inits['et_bin'] == iet
    for ieta in best_inits['eta_bin'].unique():
        ieta_mask   = best_inits['eta_bin'] == ieta
        for tag, midx in zip(best_inits['train_tag'].unique(), best_inits['model_idx'].unique()):
            model_mask = best_inits['model_idx'] == midx
            tag_mask   = best_inits['train_tag'] == tag

            full_mask = iet_mask & ieta_mask & model_mask & tag_mask
            print(iet, ieta, tag, midx, best_inits.loc[full_mask].shape)

In [28]:
best_inits[(best_inits.train_tag == 'v1.r0.mlp2') & (best_inits.et_bin == 2.) & (best_inits.eta_bin == 0.)]

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
8090,v1.r0.mlp2,2,0,0,0,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.963473,...,21991,231543,2179,1974,2200,23154,21777,19489,21991,231543
8089,v1.r0.mlp2,2,0,0,0,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.963473,...,21991,231543,2173,1697,2200,23154,21718,16771,21991,231543
8088,v1.r0.mlp2,2,0,0,0,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.963473,...,21991,231543,2173,1697,2200,23154,21717,16710,21991,231543
8091,v1.r0.mlp2,2,0,0,0,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.963473,...,21991,231543,2180,2025,2200,23154,21790,20373,21991,231543
8126,v1.r0.mlp2,2,0,0,1,3,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.964069,...,21991,231543,2178,2021,2199,23155,21777,19305,21991,231543
8125,v1.r0.mlp2,2,0,0,1,3,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.964069,...,21991,231543,2172,1695,2199,23155,21718,16705,21991,231543
8124,v1.r0.mlp2,2,0,0,1,3,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.964069,...,21991,231543,2172,1695,2199,23155,21717,16626,21991,231543
8127,v1.r0.mlp2,2,0,0,1,3,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.964069,...,21991,231543,2179,2108,2199,23155,21790,20117,21991,231543
8182,v1.r0.mlp2,2,0,0,2,1,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.962675,...,21991,231543,2178,2378,2199,23155,21777,19644,21991,231543
8181,v1.r0.mlp2,2,0,0,2,1,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.962675,...,21991,231543,2172,1982,2199,23155,21718,16617,21991,231543


When we filter sorts we must to have only one entry since.

In [12]:
best_sorts = m_cv.filter_sorts( best_inits , 'max_sp_op')
print(len(best_sorts))

240


In [13]:
best_sorts

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
994,v1.r0.mlp2,0,0,0,3,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.943268,...,28455,215903,2783,2248,2846,21590,27827,23692,28455,215903
993,v1.r0.mlp2,0,0,0,3,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.943268,...,28455,215903,2799,2685,2846,21590,27987,27231,28455,215903
992,v1.r0.mlp2,0,0,0,3,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.943268,...,28455,215903,2799,2685,2846,21590,27987,27231,28455,215903
995,v1.r0.mlp2,0,0,0,3,4,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.943268,...,28455,215903,2815,3449,2846,21590,28149,34076,28455,215903
818,v1.r0.mlp3,0,0,1,7,3,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.943981,...,28455,215903,2782,2150,2845,21591,27827,22707,28455,215903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4531,v1.r0.mlp4,2,4,2,8,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,vloose,0.989801,...,147,16216,14,33,14,1622,143,572,147,16216
4010,v1.r0.mlp5,2,4,3,8,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,loose,0.989491,...,147,16216,14,34,14,1622,143,603,147,16216
4009,v1.r0.mlp5,2,4,3,8,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,medium,0.989491,...,147,16216,14,34,14,1622,142,590,147,16216
4008,v1.r0.mlp5,2,4,3,8,2,/home/micael/Documents/NeuralRinger/Jpsiee/tun...,0,tight,0.989491,...,147,16216,14,34,14,1622,142,590,147,16216


## Get the cross-validation table

In [19]:
for op_name in op_names:
    table = best_inits.loc[best_inits.op_name == op_name]
    output = './cv_v1.r0_%s.pdf'%op_name #os.path.join(analysis_path, 'v1/r0/cv_pdfs/cv_v1.r0_%s.pdf'%op_name)
    m_cv.dump_beamer_table(table, output, tags=['v1.r0.mlp2'],# 'v1.r0.mlp5'], 
                           title='data17_13TeV %s v1 tuning (with tanh)'%op_name, 
                           cv_tag_name='Ringer (2017)',
                           tuning_ref_name='Cut-Based (2017)'
                           )

2022-01-09 21:14:05,607 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file ./cv_v1.r0_tight.pdf latex code...
2022-01-09 21:14:06,482 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file ./cv_v1.r0_medium.pdf latex code...
2022-01-09 21:14:07,485 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file ./cv_v1.r0_loose.pdf latex code...
2022-01-09 21:14:08,567 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file ./cv_v1.r0_vloose.pdf latex code...


## Plot monitoring training curves

In [38]:
aux_init = best_inits.loc[best_inits.op_name == 'tight']
aux_sort = best_sorts.loc[best_sorts.op_name == 'tight']
m_cv.plot_training_curves( aux_init, aux_sort , 'monitoring_curves' )

# Bin resume

Each bin has 10 models:
1. monitoring curves;
2. roc curve;
3. histogram.

create a for each model to plot that

In [14]:
def calc_sp(det, fake):
    return np.sqrt( np.sqrt(det*(1-fake)) * (0.5*(det+(1 - fake))) )

def plot_loss_function(loss, val_loss, best_epoch, axis):
    #  'solid', 'dashed', 'dashdot', 'dotted'
    axis.plot(loss,       color='black', ls='dashdot', lw=1.2, label='Training Loss')
    axis.plot(val_loss,   color='red', ls='solid', lw=1.2, label='Test Loss')
    axis.axvline(x=best_epoch, color='black', ls='dotted', lw=1.2, label='Best epoch')
    axis.set_ylabel('Loss Function')
    axis.set_xlabel('# epochs')
    axis.legend(loc='best')
    axis.grid()
    axis.set_title('Loss Function Evolution')

    
def plot_det(det, best_epoch, axis):
    #  'solid', 'dashed', 'dashdot', 'dotted'
    axis.plot(det,       color='dodgerblue', ls='dashdot', lw=1.2, label='Test Set')
    axis.axvline(x=best_epoch, color='black', ls='dotted', lw=1.2, label='Best epoch')
    axis.set_ylabel('Detection')
    axis.set_xlabel('# epochs')
    axis.legend(loc='best')
    axis.grid()
    axis.set_title('Detection Evolution (Test set)')

def plot_fake(fake, best_epoch, axis):
    #  'solid', 'dashed', 'dashdot', 'dotted'
    axis.plot(fake,       color='darkred', ls='dashdot', lw=1.2, label='Test Set')
    axis.axvline(x=best_epoch, color='black', ls='dotted', lw=1.2, label='Best epoch')
    axis.set_ylabel('Fake Rate')
    axis.set_xlabel('# epochs')
    axis.legend(loc='best')
    axis.grid()
    axis.set_title('Fake Rate Evolution (Test set)')

def plot_sp(sp_index, best_epoch, axis):
    #  'solid', 'dashed', 'dashdot', 'dotted'
    axis.plot(sp_index,       color='forestgreen', ls='dashdot', lw=1.2, label='Test Set')
    axis.axvline(x=best_epoch, color='black', ls='dotted', lw=1.2, label='Best epoch')
    axis.set_ylabel(r'$SP$ Index')
    axis.set_xlabel('# epochs')
    axis.legend(loc='best')
    axis.grid()
    axis.set_title(r'$SP$ Index Evolution (Test set)')

def plot_roc_curve(roc_fa, roc_pd, reference_dict, axis, ylim=[75, 105], xlim=[-.5, 50]):

    axis.plot(roc_fa*100, roc_pd*100, color='black', linewidth=1.5, ls='dashed')
    axis.set_ylabel('Probability of Dectection [%]')
    axis.set_xlabel('Fake Rate [%]')

    sp_max_index = np.argmax(calc_sp(roc_pd, roc_fa))

    axis.plot(roc_fa[sp_max_index]*100, roc_pd[sp_max_index]*100, '*', 
                 label=r'$SP_{max}$ -> $P_D$: %1.2f | $F_R$: %1.2f' %(roc_pd[sp_max_index]*100, roc_fa[sp_max_index]*100),
                 markersize=10, color=plt.cm.tab10(4))

    colors = iter([plt.cm.tab10(i) for i in range(4)])
    for iop, imarker in zip(['tight', 'medium', 'loose', 'vloose'], ['^', '<', '>', 'v']):
        m_color   = next(colors)
        pd_ref    = reference_dict[iop]['pd_ref'][0]*100
        fa_ref    = reference_dict[iop]['fa_ref'][0]*100
        pd_ringer = reference_dict[iop]['pd_val'][0]*100
        fa_ringer = reference_dict[iop]['fa_val'][0]*100
        # plot ringer adjusted
        axis.plot(fa_ringer, pd_ringer, imarker,
                  label=r'%s - OP -> $P_D$: %1.2f | $F_R$: %1.2f' %(iop, pd_ringer, fa_ringer), color=m_color, markersize=10)
        # plot cut-based reference
        axis.plot(fa_ref, pd_ref, imarker, fillstyle='none',
                  label=r'%s - OP -> Ref. $P_D$: %1.2f | $F_R$: %1.2f' %(iop, pd_ref, fa_ref), color=m_color, markersize=10)
    axis.set_ylim(ylim)
    axis.set_xlim(xlim)
    axis.grid()
    axis.legend(loc='best')
    axis.set_title('ROC Curve')

def plot_nn_histogram(hist_dict, reference_dict, axis):
        axis.hist(hist_dict['val_bkg'][1][:-1], hist_dict['val_bkg'][1], weights=hist_dict['val_bkg'][0], histtype='bar', 
                     color='slategray', lw=1.5, 
                     label='jets', alpha=.5)
        axis.hist(hist_dict['val_sgn'][1][:-1], hist_dict['val_sgn'][1], weights=hist_dict['val_sgn'][0], histtype='bar', 
                     color='blue', lw=1.5, 
                     label='electron', alpha=.5)

        colors = iter([plt.cm.tab10(i) for i in range(4)])
        for iop in ['tight', 'medium', 'loose', 'vloose']:
            axis.axvline(x=reference_dict[iop]['threshold_val'], ls='solid',
                     label=r'%s - OP' %(iop), color=next(colors), lw=1.5)
        axis.set_title('NN Output Histogram')
        axis.legend(loc='best')
        axis.grid()
        axis.set_yscale('log')
        axis.set_ylabel('Counts')
        axis.set_xlabel('NN Output')

def plot_training_resume(dataframe, output_path, et_bin=2, eta_bin=0, train_tag='v1.r0.mlp2', roc_xlim=[-.5, 35], roc_ylim=[75, 105]):
    from Gaugi import load as gload
    from kolmov.utils.constants import str_etbins_jpsiee, str_etabins
    
    # first filter by et_bin and eta_bin
    df = dataframe.loc[(dataframe.et_bin==et_bin) & (dataframe.eta_bin == eta_bin) & (dataframe.train_tag == train_tag)]
    # create a folder to save the figures
    os.makedirs(os.path.join(output_path, 'train_resume_et%i_eta%i' %(et_bin, eta_bin)), exist_ok=True)
    o_path = os.path.join(output_path, 'train_resume_et%i_eta%i' %(et_bin, eta_bin))
    # let's moving per sort
    for isort in df.sort:
        fig, ax = plt.subplots(nrows=2, ncols=3, sharex=False, sharey=False, figsize=(20,10))
        axes = ax.flatten()
        aux_df = df.loc[df.sort == isort]
        # get the file
        if len(aux_df.file_name.unique()) == 1:
            f_     = gload(aux_df.file_name.unique()[0])['tunedData'][0]
        else:
            print('something strange, must be one unique file')
        hist_dict        = f_['history']['summary']['hists']
        roc_pd, roc_fa   = f_['history']['summary']['rocs']['roc_val']

        best_epoch       = f_['history']['max_sp_best_epoch_val'][-1]
        loss             = f_['history']['loss']
        val_loss         = f_['history']['val_loss']

        sp_index         = f_['history']['max_sp_val']
        det              = f_['history']['max_sp_pd_val']
        fake             = f_['history']['max_sp_fa_val']

        # plot loss
        plot_loss_function(loss, val_loss, best_epoch, axis=axes[0])

        # plot roc curve
        plot_roc_curve(roc_fa=roc_fa, roc_pd=roc_pd, reference_dict=f_['history']['reference'], 
                       axis=axes[1], xlim=roc_xlim, ylim=roc_ylim)
        # plot nn output 
        plot_nn_histogram(hist_dict, reference_dict=f_['history']['reference'], axis=axes[2])
        # plot the detection
        plot_det(det, best_epoch=best_epoch, axis=axes[3])
        # plot sp index
        plot_sp(sp_index=sp_index, best_epoch=best_epoch, axis=axes[4])
        # plot fake
        plot_fake(fake=fake, best_epoch=best_epoch, axis=axes[5])

        fig.suptitle(r'Train sort: %i -> $E_T \in$ %s | $\eta \in$ %s' %(isort+1, str_etbins_jpsiee[et_bin], str_etabins[eta_bin]), fontsize=15)
        plt.savefig(os.path.join(o_path, 'train_resume_sort%i_et%i_eta%i.png' %(isort, et_bin, eta_bin)), dpi=150, facecolor='white', bbox_inches='tight')
        plt.close()

In [15]:
for iet, ieta in tqdm.tqdm(list(itertools.product([2], range(5)))):
#for iet, ieta in tqdm.tqdm(list(itertools.product([0], [0]))):
    plot_training_resume(best_inits, output_path=os.path.join(analysis_path, 'v1/r0/plots'), et_bin=iet, eta_bin=ieta, roc_xlim=[-.25, 100])

100%|██████████| 5/5 [07:13<00:00, 86.61s/it]


# Test on new data

In this step I'll test these models using another dataset that wasn't used durint training process. This dataset is the colision data obtained by during the 2018 year.

The data was obtained from a specific derivation for $e/\gamma$ studies. The signal are composed by electrons from $J/\Psi$ meson decay. To select this sample the _Tag and Probe_ methoed was used. This method allow us to select unbiased electrons in this dacay (probe) to use in training/test processes.

The expected output for this part is:

1. A table contains the efficiency in 2018 data comparing with cut-based (operating in 2018) and the training values (Ringer and cu-based).

In [21]:
datapath = os.path.join(l_path, 'data/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM2.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.15bins/' +\
                                                 'data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM2.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.15bins_et{ET}_eta{ETA}.h5')
test_paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(3)]

In [17]:
m_tag = 'v1.r0.mlp2'
# filter tables to the wanted tuning model
best_inits = best_inits.loc[best_inits.train_tag == m_tag]
best_sorts = best_sorts.loc[best_sorts.train_tag == m_tag]

In [18]:
def my_data_generator( path, pidname='el_lhmedium' ):  
    import pandas as pd
    from kepler import load_hdf
    df = load_hdf(path)
    # NOTE: Offline filter lhvloose -> lhmedium (as the training procedure)
    df = df.loc[ ((df[pidname]==True) & (df.target==1.0)) | ((df.target==0) & (df['el_lhvloose']==0)) ]
    return df


def my_input_generator( df ):
    '''
    This function will generate the input for ringer models.

    Arguments:
    df: the dataframe
    '''
    col_names= ['trig_L2_cl_ring_%d'%i for i in range(100)]
    rings = df[col_names].values.astype(np.float32)
    
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    rings = norm1(rings)
    return [rings]


def my_dec_generator( row, df ):
    '''
    This function will load a model from dataframe get their decision and return.

    Arguments:
    row: a dataframe row (used by kolmov)
    df:  the dataframe that will be use to get the decision
    '''
    from Gaugi import load as gload
    file_name  = row.file_name.values[0]
    i_init     = row.init.values[0]
    i_sort     = row.sort.values[0]
    model_idx  = row.model_idx.values[0]
    op_name    = row.op_name.values[0]
    tuned_data = gload(file_name)['tunedData']
    for ituned in tuned_data:
        if ((ituned['imodel'] == model_idx)\
            and (ituned['sort'] == i_sort) \
            and (ituned['init'] == i_init)):
            sequence, weights = ituned['sequence'], ituned['weights']
            thr = ituned['history']['reference']['%s_cutbased' %(op_name)]['threshold_op']
    model = tf.keras.models.model_from_json( json.dumps(sequence, separators=(',', ':')) ) #custom_objects={'RpLayer':RpLayer} )
    model.set_weights( weights )
    
    output = model.predict( my_input_generator(df), batch_size=4096 ).flatten()
    dec = np.greater(output, thr)
    return dec

In [22]:
test_table = m_cv.evaluate( best_sorts , test_paths, my_data_generator, my_dec_generator)

Fitting... :   0%|                             | 0/15 [00:00<?, ?it/s]

In [1]:
test_table

NameError: name 'test_table' is not defined